# Vectorizing

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from helpers import dump_pickle, load_pickle

In [5]:
df = load_pickle('tass/dfLemmas')

In [4]:
df.head()

,filename,category,url,text,length,trimmed,stopped,lemmas
0,13640047.txt,obschestvo,tass.ru/obschestvo/13640047,Минздрав Бразилии считает тревожной ситуацию с...,1688,минздрав бразилии считает тревожной ситуацию с...,минздрав бразилии считает тревожной ситуацию ...,минздрав бразилия считать тревожный ситуация р...
1,13633917.txt,obschestvo,tass.ru/obschestvo/13633917,На Филиппинах началась вакцинация от коронавир...,1892,на филиппинах началась вакцинация от коронавир...,филиппинах началась вакцинация коронавируса ...,филиппины начинаться вакцинация коронавирус ре...
2,13680263.txt,obschestvo,tass.ru/obschestvo/13680263,"Песков подтвердил, что Макрон перед переговора...",1771,песков подтвердил что макрон перед переговорам...,песков подтвердил макрон переговорами путин...,песков подтверждать макрон переговоры путин от...
3,13649077.txt,obschestvo,tass.ru/obschestvo/13649077,В Петербурге развернули дополнительно 200 коек...,1911,в петербурге развернули дополнительно коек...,петербурге развернули дополнительно коек ...,петербург развертывать дополнительно койка лен...
4,13656631.txt,obschestvo,tass.ru/obschestvo/13656631,Раскрываемость убийств в Татарстане достигла с...,1502,раскрываемость убийств в татарстане достигла с...,раскрываемость убийств татарстане достигла са...,раскрываемость убийство татарстан достигать са...


In [17]:
CODES = {
    'obschestvo': 0,
    'ekonomika': 1,
    'politika': 2,
    'sport': 3,
    'kultura': 4
}

df['code'] = df['category'].apply(lambda x: CODES.get(x))

In [18]:
data_train, data_test, labels_train, labels_test = train_test_split(
    df['lemmas'], 
    df['code'], 
    test_size=0.15, 
    random_state=26
)

## Settings
* `ngram_range = (1, 2)`: define unigrams, bigrams
* `max_df = 1.0`: use all terms, even those that are used in 100% of documents
* `min_df = 10`: ignore terms that are used in less than 10 documents
* `max_features`: use top-700 most frequently used terms

In [43]:
TfidfVectorizer?

In [42]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1, 2),
                        stop_words=None,
                        lowercase=False,
                        max_df=1.0,
                        min_df=10,
                        max_features=700,
                        norm='l2',
                        sublinear_tf=True)

features_train = tfidf.fit_transform(data_train).toarray()
features_train.shape

(8980, 700)

In [38]:
features_test = tfidf.transform(data_test).toarray()
features_test.shape

(1585, 700)

In [39]:
for category, code in sorted(CODES.items()):
    features_chi2 = chi2(features_train, labels_train == code)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names_out())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    top_unigrams = ', '.join(unigrams[-5:])
    top_bigrams = ', '.join(bigrams[-5:])

    print(f'Category {category}')
    print(f'\tUnigrams: {top_unigrams}')
    print(f'\tBigrams: {top_bigrams}')
    print('\n')

Category ekonomika
	Unigrams: объем, цена, газ, рынок, компания
	Bigrams: игра пекин, январь год, сборная россия, прошлое год, олимпийский игра


Category kultura
	Unigrams: музей, фестиваль, режиссер, театр, фильм
	Bigrams: мид рф, санкт петербург, игра пекин, сборная россия, олимпийский игра


Category obschestvo
	Unigrams: заражение, заболеваемость, сутки, инфекция, пациент
	Bigrams: игра пекин, сборная россия, олимпийский игра, коронавирусный инфекция, случай заражение


Category politika
	Unigrams: песков, безопасность, нато, лавров, мид
	Bigrams: пресс секретарь, министр иностранный, иностранный дело, гарантия безопасность, мид рф


Category sport
	Unigrams: олимпиада, пекин, сборная, игра, олимпийский
	Bigrams: командный турнир, олимпиада пекин, игра пекин, сборная россия, олимпийский игра




In [40]:
bigrams

['сообщать пресс',
 'принимать решение',
 'генеральный директор',
 'случай заражение',
 'санкт петербург',
 'принимать участие',
 'декабрь год',
 'пресс конференция',
 'настоящий время',
 'коронавирусный инфекция',
 'пресс служба',
 'январь год',
 'прошлое год',
 'февраль год',
 'рф владимир',
 'дмитрий песков',
 'президент россия',
 'министр иностранный',
 'гарантия безопасность',
 'пресс секретарь',
 'иностранный дело',
 'владимир путин',
 'президент рф',
 'мид рф',
 'такой мнение',
 'второй место',
 'командный турнир',
 'олимпиада пекин',
 'игра пекин',
 'сборная россия',
 'олимпийский игра']

In [85]:
dump_pickle('tass/dfCoded', df)
dump_pickle('tass/tfidf', tfidf)
dump_pickle('tass/tfidf2', tfidf2)
dump_pickle('tass/data_train', data_train)
dump_pickle('tass/data_test', data_test)
dump_pickle('tass/features_train', features_train)
dump_pickle('tass/features_test', features_test)
dump_pickle('tass/labels_train', labels_train)
dump_pickle('tass/labels_test', labels_test)

## TF-IDF with max value

In [44]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1, 2),
                        stop_words=None,
                        lowercase=False,
                        max_df=0.95,
                        min_df=0.03,
                        max_features=700,
                        norm='l2',
                        sublinear_tf=True)

features_train = tfidf.fit_transform(data_train).toarray()
features_train.shape

(8980, 700)

In [45]:
features_test = tfidf.transform(data_test).toarray()
features_test.shape

(1585, 700)

In [46]:
for category, code in sorted(CODES.items()):
    features_chi2 = chi2(features_train, labels_train == code)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names_out())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    top_unigrams = ', '.join(unigrams[-5:])
    top_bigrams = ', '.join(bigrams[-5:])

    print(f'Category {category}')
    print(f'\tUnigrams: {top_unigrams}')
    print(f'\tBigrams: {top_bigrams}')
    print('\n')

Category ekonomika
	Unigrams: объем, цена, газ, рынок, компания
	Bigrams: январь год, сборная россия, прошлое год, олимпийский игра, составлять рубль


Category kultura
	Unigrams: музей, фестиваль, режиссер, театр, фильм
	Bigrams: пресс служба, санкт петербург, игра пекин, сборная россия, олимпийский игра


Category obschestvo
	Unigrams: заражение, коронавирус, сутки, инфекция, пациент
	Bigrams: игра пекин, сборная россия, олимпийский игра, коронавирусный инфекция, случай заражение


Category politika
	Unigrams: песков, безопасность, лавров, нато, мид
	Bigrams: пресс секретарь, министр иностранный, иностранный дело, гарантия безопасность, мид рф


Category sport
	Unigrams: олимпиада, пекин, сборная, игра, олимпийский
	Bigrams: завершаться февраль, олимпиада пекин, игра пекин, сборная россия, олимпийский игра




In [47]:
bigrams

['сообщать пресс',
 'принимать решение',
 'генеральный директор',
 'случай заражение',
 'санкт петербург',
 'принимать участие',
 'декабрь год',
 'пресс конференция',
 'настоящий время',
 'коронавирусный инфекция',
 'пресс служба',
 'январь год',
 'прошлое год',
 'февраль год',
 'российский федерация',
 'составлять рубль',
 'дмитрий песков',
 'рф владимир',
 'премьер министр',
 'президент россия',
 'министр иностранный',
 'гарантия безопасность',
 'пресс секретарь',
 'иностранный дело',
 'владимир путин',
 'президент рф',
 'мид рф',
 'первый место',
 'такой мнение',
 'второй место',
 'завершаться февраль',
 'олимпиада пекин',
 'игра пекин',
 'сборная россия',
 'олимпийский игра']

In [48]:
dump_pickle('tass/max_df/dfCoded', df)
dump_pickle('tass/max_df/tfidf', tfidf)
dump_pickle('tass/max_df/data_train', data_train)
dump_pickle('tass/max_df/data_test', data_test)
dump_pickle('tass/max_df/features_train', features_train)
dump_pickle('tass/max_df/features_test', features_test)
dump_pickle('tass/max_df/labels_train', labels_train)
dump_pickle('tass/max_df/labels_test', labels_test)